# Data Pre-processing

Using the Titanic dataset from [this](https://www.kaggle.com/c/titanic/overview) Kaggle competition.

This dataset contains information about 891 people who were on board the ship when departed on April 15th, 1912. As noted in the description on Kaggle's website, some people aboard the ship were more likely to survive the wreck than others. There were not enough lifeboats for everybody so women, children, and the upper-class were prioritized. Using the information about these 891 passengers, the challenge is to build a model to predict which people would survive based on the following fields:

- **Name** (str) - Name of the passenger
- **Pclass** (int) - Ticket class
- **Sex** (str) - Sex of the passenger
- **Age** (float) - Age in years
- **SibSp** (int) - Number of siblings and spouses aboard
- **Parch** (int) - Number of parents and children aboard
- **Ticket** (str) - Ticket number
- **Fare** (float) - Passenger fare
- **Cabin** (str) - Cabin number
- **Embarked** (str) - Port of embarkation (C = Cherbourg, Q = Queenstown, S = Southampton)

In [ ]:
# Importing libraries

import matplotlib.pyplot as plt
import numpy as np
import ? as pd # ? pandas
import seaborn as sns #built on top of matplotlib so need to load matplotlib

In [ ]:
# Reading the dataset

titanic = pd.read_csv('titanic.csv')
# titanic.head()

In [ ]:
# Checking the number of rows and columns ? shape

titanic.?

In [ ]:
# Checking column names

titanic.columns

In [ ]:
# Checking columns with missing values ? sum()

titanic.isnull().?

In [ ]:
titanic.info()

# Handling missing values

In real world, the datasets are not clean and may have missing values. We must know how to find and deal with these missing values. 

One of the strategies to deal with missing values is to remove them from the dataset. 

However, whenever possible, we would like to impute the data, which is a process of filling the missing values. 

Let us take a look at the example on how to handle missing values. Only 3 columns out of 12 contain missing data. 

The Age column contains about 20% missing data, which can be dealt with using a technique called imputation, which means replacing the missing values with a known value, such as the mean, median or mode. Age is quantitative, so either mean or median imputation can be done. Embarked is a categorical variable, so mode imputation can be done.

As for the Cabin variable containing 77% missing data, we can simply delete the column. More sophisticated methods would be to see how Cabin numbers relate to the Survived or Pclass column and deal with it accordingly. For example, some of the cabins could be located near where the lifeboats are stored, so passengers residing in those cabins could have a higher chance of survival.

In [ ]:
# Is dropping all the rows with missing values a good idea? dropna()
new_titanic = titanic.?

In [ ]:
# Checking if all the missing values are removed

new_titanic.isnull().?

In [ ]:
# Checking the shape of the new dataset after dropping

print("No. of rows and columns:",new_titanic.?) 


## Mode Imputation

In [ ]:
# titanic.pivot_table('PassengerId', index = 'Survived', columns = 'Embarked', aggfunc='count')
mode_embarked = titanic['Embarked'].mode()[0] # Python still thinks the mode-aggregated object is a DataSeries (one column of a DataFrame), so we need to select the string inside the DataSeries, hence the [0].


In [ ]:
titanic['Embarked'].fillna(mode_embarked,inplace = ?) #to replace the existing dataset in fillna use attribute inplace = True

In [ ]:
# Checking the shape of the new dataset after replacing missing values

print("No. of rows and columns:",titanic.shape)

In [ ]:
# Checking if the missing values are removed from Embarked

print(titanic.isnull().sum())

## Mean Imputation

In [ ]:
# Checking the mean age of passengers groupby Survived or not ? mean()

titanic.groupby('Survived')['Age'].?

In [ ]:
titanic['Age'].mean() # Mean age

In [ ]:
# Replacing missing values in the Age column by mean age of the passengers irrespective of survived or not

titanic['Age'].fillna(titanic['Age'].mean(), inplace = ?) # ?

In [ ]:
# Checking the shape of the new dataset after replacing missing values

print("No. of rows and columns:",titanic.shape)

In [ ]:
# Checking if the missing values are removed from Embarked

print(titanic.isnull().sum())

## Indicator Variable

In [ ]:
# Checking if there is a pattern between Cabin missing and Survived

titanic.groupby(titanic['Cabin'].isnull())['Survived'].mean() # those who died had most missing value of cabin i.e., only 30% of those with missing Cabin value Survived 

In [ ]:
titanic['Cabin'].isnull() # to see the true/ false value for cabin is null

In [ ]:
titanic['Cabin'] # to see the actual cabin values


In [ ]:
# Creating a new indicator variable named 'Cabin_ind'

titanic['Cabin_ind'] = np.where(titanic['Cabin'].isnull(),'0','1')

In [ ]:
# Check to see new column is added

titanic.isnull().sum()

## Deletion

The remaining column with missing data is Cabin (the cabin number). This column contains 77% missing data, so the easiest method would be to get rid of it entirely. Columns can be deleted using the .drop() method


In [ ]:
titanic.drop(columns='Cabin', inplace=True)

In [ ]:
# Check column Cabin is removed

titanic.isnull().sum()

Additionally, the columns PassengerId, Name, Ticket are going to be irrelevant for further analysis, so we can drop those as well.

In [ ]:
titanic.drop(columns=['PassengerId','Name','Ticket'], inplace=True)

In [ ]:
# Check columns are removed

titanic.isnull().sum()

In [ ]:
titanic.info()

## Creating Dummy Variables

We now have 6 numerical columns and three categorical column (Sex, Embarked, Cabin_ind). 
Also, we need to realize that Pclass is an ordinal categorical variable, with 1st class having a higher status than 2nd class, and so on. These categorical varibales need to be changed into numerics because the machine learning algorithm can only understand numbers.

In the case of Pclass, the classes are already represented as numbers but in the reverse order. 3rd class is represented as 3 even though it signifies a lower status than 1st class, which is represented as 1. We could reverse the order and make 3rd class 1 and 1st class 3 but it would be misleading and confusing to interpret.

Representing ordinal categorical variables as integers depending on the order of importance assigned to them is known as label encoding. Label encoding can also be used for nominal variables which has no inherent order, for example, Red, Green, Blue being encoded as 1, 2, 3. The disadvantage is that the machine learning algorithm would misinterpret Blue to have a higher quantitative weight than Red even though they are supposed to be equally important.

Therefore, we use one-hot encoding for Pclass, Sex, Embarked and Cabin_ind. One-hot encoding separates categories into binary values of 0 and 1.


Consider the Sex column first. Initially, it contains the categories "Male" and "Female", specifying the sex of the passenger. We would like to encode these categories as numbers instead of letters so we apply the pandas method .get_dummies() onto the Sex column. After applying the .get_dummies() method, we see two new columns Sex_female and Sex_male, and the original Sex column has disappeared.

In the Sex_male column, if the passenger is male, then he is encoded as 1 and if not she is encoded as 0. The same thing is repeated for the Sex_female column. However, this repetition is undesirable to have because all the required information is already captured within one column. Either keep the Sex_male column and drop the Sex_female, or keep the Sex_female and drop the Sex_male.

In [ ]:
titanic = pd.?(titanic, columns=['Sex']) # ? get_dummies

In [ ]:
titanic.head()

In [ ]:
# Drop Sex_female
titanic.drop('Sex_female', axis=?, inplace=True) # axis=1 specifies that a column is being dropped. If we want to drop rows, we specify axis=0


In [ ]:
titanic.head()

We perform the dummification process of getting dummy columns (the Sex_male and Sex_female are called dummy variables, which are obtained from the original Sex column) for the Pclass column. This time, we add an additional argument drop_first=True to the get_dummies() method to drop one irrelevant column:

In [ ]:
titanic = pd.get_dummies(titanic, columns=['Pclass'], drop_first=?) # ? True

In [ ]:
titanic.head()

In the Pclass column, we had 3 categories: first, second and third class passengers. One-hot encoding for 3 categories works like this: if the passenger is in 1st class, Pclass_1 = 1 and Pclass_2 = Pclass_3 = 0. If the passenger is in 2nd class, Pclass_2 = 1 and Pclass_1 = Pclass_3 = 0, and similarly for 3rd class passengers.

In this case, all the information is captured in two columns (the irrelevant column was already dropped by specifying the drop_first=True argument in the previous line of code). Likewise, if we have 4 categories in a column, we create 3 dummies and drop one, and so on.


In [ ]:
#Create dummies for the other two qualitative variables
titanic = pd.get_dummies(titanic, columns = ['Embarked','Cabin_ind'],drop_first=True)

In [ ]:
titanic.head()

In [ ]:
titanic.info()

## Feature Engineering

In [ ]:
titanic['Family_Cnt'] = titanic['SibSp'] + titanic['Parch'] # already discussed in previous tutorial on exploratory data analysis
sns.catplot(data = titanic,
            x = 'Family_Cnt',
            y= 'Survived',
            kind = 'point',
            aspect = 2,
            ci = None)

In [ ]:
# Can drop SibSp and Parch from dataset since we wont use them for further analysis

titanic.drop(columns=['SibSp','Parch'], inplace=True)

In [ ]:
titanic.info()

## Handling Outlier

In [ ]:
sns.catplot(data = titanic,
            x = 'Fare',
            kind = 'box',
           whis = [5,95])

In [ ]:
help(sns.catplot)

In [ ]:
#Winsorise
from scipy.stats.mstats import winsorize

In [ ]:
# Trimming from top and bottom

titanic['New_fare'] = winsorize(titanic['Fare'], limits=[0.05, 0.05]) # how much to winsorize from top and bottom 5-95%

In [ ]:
sns.catplot(data = titanic,
            y = 'New_fare',
            kind = 'box',
           whis = [5,95]) # Brought the data in normal range 

In [ ]:
# Saving the pre-processed dataset in local drive

titanic.to_csv('titanic_cleaned_1807.csv', encoding = 'utf-8-sig') 
# files.download('titanic_cleaned_1807.csv')